# Task 1 ：复现实验

分析文章的baseline，发现该方法无法在真实场景中使用：
1. 选取的数据集中，有异常数据
2. 选取的特征中，有无效特征
3. 通过特征图筛选，可以筛选出3～4个特征，使用筛选出来的特征，可达到与全部特征相近的结果
4. 选取的特征，为不稳定特征，无法在真实场景中使用

In [3]:
import os,sys
sys.path.append("./share")
sys.path.append("./common")
sys.path.append("./ml")
from Config import g_data_root
import pandas as pd
import numpy as np
import warnings,logging,re
from SHVisualize import CSHVisualize
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from datetime import datetime
from SHModelClassify import CSHModelClassify
from SHSample import CSHSample
from SHDataProcess import CSHDataProcess
from SHDataEDA import CSHDataDistribution,CSHDataTest
from SHFeatureSelect import CSHFeature
from IoTNoise import CIoTNoise
import seaborn as sns
import h2o
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
tqdm.pandas()
import warnings
warnings.simplefilter(action='ignore')
h2o.init(nthreads = -1, verbose=False)
np.random.seed(42)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.4
H2O_cluster_version_age:,3 months and 4 days
H2O_cluster_name:,H2O_from_python_root_sden0h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,19.45 Gb
H2O_cluster_total_cores:,20
H2O_cluster_allowed_cores:,20
H2O_cluster_status:,"locked, healthy"


In [4]:
def is_valid_ip(ip_str):
    ip_pattern = re.compile(r'^((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$')
    if ip_pattern.match(ip_str):
        octets = ip_str.split('.')
        for octet in octets:
            if int(octet) > 255:
                return False
        return True
    else:
        return False

def is_valid_time(time_str):
    tmp = time_str.strip()
    tmp = tmp[:-3]
    try:
        datetime.strptime(tmp, '%Y %H:%M:%S.%f')
        return True
    except ValueError:
        return False

# 定义公共函数

In [5]:
df = pd.read_csv("%s/raw/Selected dataset for ML and DL/ML-EdgeIIoT-dataset.csv"%g_data_root,low_memory=False)

FileNotFoundError: [Errno 2] No such file or directory: './IIoTSet//raw/Selected dataset for ML and DL/ML-EdgeIIoT-dataset.csv'

# 查看样本格式

# 过滤掉异常数据
1. 时间格式错误
2. ip地址错误

In [ ]:
def pre_process(x):
    ret = "ok"
    if not is_valid_time(x['frame.time']):
        ret = "frame.time"
    elif not is_valid_ip(x['ip.src_host']) and not is_valid_ip(x['arp.src.proto_ipv4']):
        ret = "src_host"
    elif not is_valid_ip(x['ip.dst_host']) and not is_valid_ip(x['arp.dst.proto_ipv4']):
        ret = "dst_host"
    return ret
    
df['flag'] = df.progress_apply(pre_process,axis=1)
df = df[df['flag'] == "ok"]

del df['frame.time']
del df['ip.src_host']
del df['ip.dst_host']
del df['arp.src.proto_ipv4']
del df['arp.dst.proto_ipv4']
del df['tcp.srcport']
del df['tcp.dstport']
del df['udp.port']
del df['tcp.payload']
del df['http.tls_port']
del df['http.file_data']
del df['icmp.transmit_timestamp']
del df['http.request.uri.query']
del df['http.referer']
del df['http.request.full_uri']
del df['http.request.version']
del df['tcp.options']
del df['dns.qry.name.len']
del df['mqtt.msg']

one_hot_encoded = pd.get_dummies(df['http.request.method'],prefix='http.request.method')
df = pd.concat([df, one_hot_encoded], axis=1)
#one_hot_encoded = pd.get_dummies(df['mqtt.conack.flags'],prefix='mqtt.conack.flags')
#df = pd.concat([df, one_hot_encoded], axis=1)
#one_hot_encoded = pd.get_dummies(df['mqtt.protoname'],prefix='mqtt.protoname')
#df = pd.concat([df, one_hot_encoded], axis=1)
#one_hot_encoded = pd.get_dummies(df['mqtt.topic'],prefix='mqtt.topic')
#df = pd.concat([df, one_hot_encoded], axis=1)

del df['http.request.method']
del df['mqtt.conack.flags']
del df['mqtt.protoname']
del df['mqtt.topic']
df['Attack_type']=df['Attack_type'].astype(dtype='category')
df['Attack_code']=df['Attack_type'].cat.codes

# 发现异常特征
1. mqtt.conack.flags
2. mqtt.protoname
3. mqtt.topic

In [ ]:
#mqtt.conack.flags_0',
#'mqtt.conack.flags_0.0',
#'mqtt.conack.flags_0x00000000'
#df_tmp = df[['http.request.method','mqtt.protoname','mqtt.topic','mqtt.conack.flags','Attack_label']]
#df_tmp=df_tmp[df_tmp['Attack_label'] == 1]
#df_tmp['mqtt.topic'].unique()

# 选取base的特征
1. 去掉冗余字段
2. 设置特征字段和标签字段
3. 去掉只有唯一值的特征

In [ ]:
df_base = df.copy(deep=True)
del df_base['Attack_type']
del df_base['flag']
del df_base['Attack_code']

#for col in df_base.columns.tolist():
#    if col in ['tcp.ack','tcp.flags','tcp.connection.rst','http.request.method_0.0','tcp.ack_raw','http.request.method_0']:
#        del df_base[col]
        
for key in df_base.columns:
    if df_base[key].nunique() <= 1:
        del df_base[key]
        
#for key,type in zip(df_base.dtypes.index,df_base.dtypes):
#    key = str(key)
#    if type == 'bool':
#        df_base[key] = df_base[key].astype(int)

col_y = "Attack_label"
col_x = []
for col in df_base.columns.tolist():
    if not col == col_y:
        col_x.append(col)

# 进行SMOTE采样

In [ ]:
#df_tmp = df_base
df_tmp = CSHSample.resample_smote(df_base,x_columns=col_x,y_column=col_y) 

# 对特征字段进行标准化处理

In [ ]:
df_data, scale_columns = CSHDataProcess.get_scale(df_tmp,y_column=col_y)
df_raw = df_data.copy(deep = True)

# 查看信息图

# 使用原始特征

In [6]:
#添加噪音的特征有哪些
dtypes_dict = df_raw.dtypes.to_dict()
noised_columns = []
for f,t in dtypes_dict.items():
    if t in ['int','int64','float','bool']:
        if f != col_y:
            noised_columns.append(f)
print("Total noised columns",len(noised_columns),len(col_x))

NameError: name 'df_raw' is not defined

## 创建数据集

In [7]:
def add_noise(df_input,ratio=0.1):
    df_data = df_input.copy(deep = True)
    for feature in df_data.keys().tolist():
        if feature in ['Attack_label']:
            continue
        dtypes_dict = df_input.dtypes.to_dict()
        type = dtypes_dict[feature]
        df_data[feature] = CIoTNoise.add_noise(feature,type,df_data,ratio)
    return df_data

def train_test(df_train,df_test):
    model = CSHModelClassify()
    model.train(df_train,x_columns=col_x,y_column=col_y,train_ratio = 0)
    df_importance = model.importance()
    df_result = pd.DataFrame()
    for ratio in tqdm([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]):
        df_test_noised = add_noise(df_test,ratio=ratio)
        df_verify_noised = model.evaluate(df_test_noised,x_columns=col_x,y_column=col_y)
        df_verify_noised['ratio'] = ratio
        df_result = pd.concat([df_result,df_verify_noised],ignore_index=True)
    df_result = df_result.reset_index(drop=True)
    return df_importance,df_result

df_total_importance = pd.DataFrame()
df_total_performance= pd.DataFrame()

## 原始数据

In [8]:
df_data = df_raw.copy(deep = True)
df_train,df_test = CSHSample.split_dataset(df_data)
df_importance,df_verify = train_test(df_train,df_test)
df_importance['kind'] = 'Session'
df_verify['kind'] = 'Session'
df_total_importance = pd.concat([df_total_importance,df_importance],ignore_index=True).reset_index(drop=True)
df_total_performance = pd.concat([df_total_performance,df_verify],ignore_index=True).reset_index(drop=True)

NameError: name 'df_raw' is not defined

## 使用Normal转换后特征

In [ ]:
df_data = CSHSample.resample_smote(df_base,x_columns=col_x,y_column=col_y) 
df_tmp = df_data.copy(deep = True)
scale_colums = []
for key,type in zip(df_tmp.keys(),df_tmp.dtypes):
    if not type in ["bool","object","category"] and not key in [col_y]:
        scale_colums.append(key)
    if df_tmp[key].dtype =='bool':
        df_tmp[key] = df_tmp[key].astype(int)
for key in scale_colums:
    df_tmp[key] = (df_tmp[key]-df_tmp[key].min())/(df_tmp[key].max()-df_tmp[key].min()) + 1
    df_tmp[key],l = CSHDataProcess.normal_transform(df_tmp[key])
    
df_train,df_test = CSHSample.split_dataset(df_tmp)
df_importance,df_verify = train_test(df_train,df_test)
df_importance['kind'] = 'Session'
df_verify['kind'] = 'Session'
df_total_importance = pd.concat([df_total_importance,df_importance],ignore_index=True).reset_index(drop=True)
df_total_performance = pd.concat([df_total_performance,df_verify],ignore_index=True).reset_index(drop=True)

## 只使用4个特征

In [ ]:
df_data = CSHSample.resample_smote(df_base,x_columns=col_x,y_column=col_y) 
df_tmp = df_data.copy(deep = True)
scale_colums = []
for key,type in zip(df_tmp.keys(),df_tmp.dtypes):
    if not type in ["bool","object","category"] and not key in [col_y]:
        scale_colums.append(key)
    if df_tmp[key].dtype =='bool':
        df_tmp[key] = df_tmp[key].astype(int)
for key in scale_colums:
    df_tmp[key] = (df_tmp[key]-df_tmp[key].min())/(df_tmp[key].max()-df_tmp[key].min()) + 1
    df_tmp[key],l = CSHDataProcess.normal_transform(df_tmp[key])
    
col_x = ['tcp.ack','tcp.flags','tcp.connection.rst','http.request.method_0.0']
col_y = 'Attack_label'
df_tmp = df_data[col_x]
df_tmp[col_y] = df_data[col_y]
df_train,df_test = CSHSample.split_dataset(df_tmp)
df_importance,df_verify = train_test(df_train,df_test)
df_importance['kind'] = 'Session'
df_verify['kind'] = 'Session'
df_total_importance = pd.concat([df_total_importance,df_importance],ignore_index=True).reset_index(drop=True)
df_total_performance = pd.concat([df_total_performance,df_verify],ignore_index=True).reset_index(drop=True)

In [12]:
df_total_importance.to_csv("./IIoTSet/result/ML-performance/task-1/importance.csv")
df_total_performance.to_csv("./IIoTSet/result/ML-performance/task-1/performance.csv")